In [1]:
# Refactoring NASEM model
import nasem_dairy as nd
import pandas as pd
import math
from pprint import pprint


# use package dir to always use /data folder regardless of where this is executed from
import importlib_resources

path_to_package_data = importlib_resources.files("nasem_dairy.data")


# Read_csv to load required data into env
user_diet_in, animal_input_in, equation_selection_in = nd.read_csv_input(path_to_package_data.joinpath("./input.csv"))

# Load feed library
feed_library_in = pd.read_csv(path_to_package_data.joinpath("NASEM_feed_library.csv"))

# Prepare infusion data (This will be optional as a default dict of 0 is provided to function otherwise)
infusion_custom = nd.read_infusion_input(path_to_package_data.joinpath("infusion_input.csv"))

# Refactor check_animal_lactation_day

In [ ]:
def calculate_An_LactDay_MlkPred(An_LactDay: int) -> int:
    """
    An_LactDay_MlkPred: An_LactDay but capped at day 375
    """
    # Cap DIM at 375 d to prevent the polynomial from getting out of range, Line 2259
    if An_LactDay <= 375:
        An_LactDay_MlkPred = An_LactDay
    elif An_LactDay > 375:
        An_LactDay_MlkPred = 375
    
    return An_LactDay_MlkPred

# Refactor calculate_Mlk_Fat_g 

In [ ]:
import numpy as np

def calculate_Trg_Mlk_Fat(Trg_MilkProd: float, Trg_MilkFatp: float) -> float:
    """
    Trg_Mlk_Fat: Target milk fat in kg, based on user input milk production and fat % 
    """
    Trg_Mlk_Fat = Trg_MilkProd * Trg_MilkFatp / 100 # Line 2262
    return Trg_Mlk_Fat

def calculate_Trg_Mlk_Fat_g(Trg_Mlk_Fat: float) -> float:
    """
    Trg_Mlk_Fat_g: Target milk fat g, based on user input milk production and fat % 
    """
    Trg_Mlk_Fat_g = Trg_Mlk_Fat * 1000  # Line 2263
    return Trg_Mlk_Fat_g

def calculate_Mlk_Fatemp_g(An_StatePhys: str, An_LactDay_MlkPred: int, Dt_DMIn: float, Dt_FAIn: float, Dt_DigC160In: float, Dt_DigC183In: float, Abs_Ile_g: float, Abs_Met_g: float):
    """
    Mlk_Fatemp_g: Milk fat prediciton, g, from Daley et al. no year given 
    """
    if An_StatePhys == "Lactating Cow":
        Mlk_Fatemp_g = 453 - 1.42 * An_LactDay_MlkPred  # Line 2259, (Equation 20-215, p. 440)
        + 24.52 * (Dt_DMIn - Dt_FAIn) 
        + 0.41 * Dt_DigC160In * 1000 
        + 1.80 * Dt_DigC183In * 1000 
        + 1.45 * Abs_Ile_g 
        + 1.34 * Abs_Met_g
    else:
        Mlk_Fatemp_g = 0    # Line 2261
    return Mlk_Fatemp_g

def calculate_Mlk_Fat_g(mFat_eqn: int, Trg_Mlk_Fat_g: float, Mlk_Fatemp_g: float) -> float:
    """
    Mlk_Fat_g: Predicted milk fat, g
    """
    Mlk_Fat_g = np.where(mFat_eqn==0, Trg_Mlk_Fat_g, Mlk_Fatemp_g)
    return Mlk_Fat_g

def calculate_Mlk_Fat(Mlk_Fat_g: float) -> float:
    Mlk_Fat = Mlk_Fat_g / 1000  # Line 2267
    return Mlk_Fat


# Refactor calculate_Mlk_Prod_comp

In [ ]:
def calculate_Mlk_NP(Mlk_NP_g: float) -> float:
    """
    Mlk_NP: Net protein in milk, kg NP/d
    """
    Mlk_NP = Mlk_NP_g / 1000    # Line 2210
    return Mlk_NP

def calculate_Mlk_Prod_comp(An_Breed: str, Mlk_NP: float, Mlk_Fat: float, An_DEIn: float, An_LactDay_MlkPred: int, An_Parity_rl: int) -> float:
    """
    Mlk_Prod_comp: Component based milk production prediciton, kg/d
    """
    # Component based milk production prediction; derived by regression from predicted milk protein and milk fat
    Mlk_Prod_comp = 4.541   # #Holstein equation, Line 2275
    + 11.13 * Mlk_NP 
    + 2.648 * Mlk_Fat 
    + 0.1829 * An_DEIn 
    - 0.06257 * (An_LactDay_MlkPred - 137.1) 
    + 2.766e-4 * (An_LactDay_MlkPred - 137.1)**2 
    + 1.603e-6 * (An_LactDay_MlkPred - 137.1)**3 
    - 7.397e-9 * (An_LactDay_MlkPred - 137.1)**4 
    + 1.567 * (An_Parity_rl - 1)

    if An_Breed == "Jersey":
        Mlk_Prod_comp = Mlk_Prod_comp - 3.400   # Line 2278
    elif (An_Breed != "Jersey") & (An_Breed != "Holstein"):
        Mlk_Prod_comp = Mlk_Prod_comp - 1.526
    return Mlk_Prod_comp


# Refactor calculate_Mlk_Prod_MPalow

In [ ]:
def calculate_An_MPavail_Milk_Trg(An_MPIn: float, An_MPuse_g_Trg: float, Mlk_MPuse_g_Trg: float) -> float:
    """
    An_MPavail_Milk_Trg: Metabolizalbe protein available for milk production, kg MP
    """
    An_MPavail_Milk_Trg = An_MPIn - An_MPuse_g_Trg / 1000 + Mlk_MPuse_g_Trg / 1000    # Line 2706
    return An_MPavail_Milk_Trg

def calculate_Mlk_NP_MPalow_Trg_g(An_MPavail_Milk_Trg: float, coeff_dict: dict) -> float:
    """
    Mlk_NP_MPalow_Trg_g: net protein available for milk production, g milk NP/d
    """
    req_coeff = ['Kx_MP_NP_Trg']
    nd.check_coeffs_in_coeff_dict(coeff_dict, req_coeff)
    Mlk_NP_MPalow_Trg_g = An_MPavail_Milk_Trg * coeff_dict['Kx_MP_NP_Trg'] * 1000   # g milk NP/d, Line 2707
    return Mlk_NP_MPalow_Trg_g

def calculate_Mlk_Prod_MPalow(Mlk_NP_MPalow_Trg_g: float, Trg_MilkTPp: float) -> float:
    """
    Mlk_Prod_MPalow: Metabolizable protein allowable milk production, kg/d
    """
    Mlk_Prod_MPalow = Mlk_NP_MPalow_Trg_g / (Trg_MilkTPp / 100) / 1000  # Line 2708, kg milk/d using Trg milk protein % to predict volume
    return Mlk_Prod_MPalow


# Refactor calculate_Mlk_Prod_NEalow

In [ ]:
def calculate_An_MEavail_Milk(An_MEIn: float, An_MEgain: float, An_MEmUse: float, Gest_MEuse: float) -> float:
    """
    An_MEavail_Milk: Metabolisable energy available milk production, Mcal/d
    """
    An_MEavail_Milk = An_MEIn - An_MEgain - An_MEmUse - Gest_MEuse  # Line 2897
    return An_MEavail_Milk


def calculate_Mlk_Prod_NEalow(An_MEavail_Milk: float, Trg_NEmilk_Milk: float, coeff_dict: dict) -> float:
    """
    Mlk_Prod_NEalow: Net energy allowable milk production, kg/d
    """
    req_coeff = ['Kl_ME_NE']
    nd.check_coeffs_in_coeff_dict(coeff_dict, req_coeff)
    Mlk_Prod_NEalow = An_MEavail_Milk * coeff_dict['Kl_ME_NE'] / Trg_NEmilk_Milk    # Line 2898, Energy allowable Milk Production, kg/d
    return Mlk_Prod_NEalow


# run_NASEM

In [2]:
def dev_run_NASEM_model(user_diet: pd.DataFrame, 
                        animal_input: dict, 
                        equation_selection: dict, 
                        feed_library_df: pd.DataFrame, 
                        coeff_dict: dict = nd.coeff_dict,
                        infusion_input: dict = nd.infusion_dict,
                        MP_NP_efficiency_input: dict = nd.MP_NP_efficiency_dict
                        ):
    """
    Run the NASEM (National Academies of Sciences, Engineering, and Medicine) Nutrient Requirements of Dairy Cattle  model.

    Parameters
    ----------
    user_diet : pd.DataFrame
        DataFrame containing user-defined diet composition.
    animal_input : dict
        Dictionary containing animal-specific input values.
    equation_selection : dict
        Dictionary containing equation selection criteria.
    feed_library_df : pd.DataFrame
        DataFrame containing the feed library data.
    coeff_dict : dict, optional
        Dictionary containing coefficients for the model, by default `nd.coeff_dict`.
    infusion_input : dict, optional
        Dictionary containing infusion input data, by default `nd.infusion_dict`.

    Returns
    -------
    Multiple
        Currently returns animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict
        To be updated

    Notes
    -----
    - 

    Examples
    --------
    Run the NASEM dairy model with user-defined inputs:

    ```python
    import nasem_dairy as nd
    nd.dev_run_NASEM_model(
        user_diet=user_diet_df,
        animal_input=animal_input_dict,
        equation_selection=equation_selection_dict,
        feed_library_df=feed_library_df
    )
    ```
    """
    ########################################
    # Step 1: Read User Input
    ########################################
    # prevent mutable changes outside of expected scope (especially for Shiny):
    user_diet = user_diet.copy()
    animal_input = animal_input.copy()

    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    list_of_feeds = user_diet['Feedstuff'].tolist()
    # feed_data = fl_get_rows(list_of_feeds, path_to_db)
    feed_data = nd.get_feed_rows_feedlibrary(list_of_feeds, feed_library_df)


    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    # list_of_feeds = user_diet['Feedstuff'].tolist()
    # feed_data = nd.fl_get_rows(list_of_feeds, '../../src/nasem_dairy/data/diet_database.db')

    feed_data = feed_data.reset_index(names='Feedstuff')

    diet_info_initial = pd.DataFrame({'Feedstuff': user_diet['Feedstuff']})
    diet_info_initial = diet_info_initial.merge(feed_data, how='left', on='Feedstuff')

    # Add Fd_DMInp to the diet_info dataframe
    Fd_DMInp = user_diet.set_index('Feedstuff')['kg_user'] / user_diet['kg_user'].sum()

    diet_info_initial.insert(1, 'Fd_DMInp', 
        Fd_DMInp.reindex(
            diet_info_initial['Feedstuff']).values)

    diet_info_initial['Fd_DMIn'] = diet_info_initial['Fd_DMInp'] * animal_input['DMI']      # Should this be done after DMI equations?

    # Add Fd_DNDF48 column, need to add to the database
    # diet_info_initial['Fd_DNDF48'] = 0

    # Calculate additional physiology values
    animal_input['An_PrePartDay'] = animal_input['An_GestDay'] - animal_input['An_GestLength']
    animal_input['An_PrePartWk'] = animal_input['An_PrePartDay'] / 7

    del (list_of_feeds, Fd_DMInp)

    # Check equation_selection to make sure they are integers.
    # This is especially important for Shiny, which may return strings
    # It's important they are correct for if statements below.
    equation_selection_in = equation_selection.copy()
    equation_selection = {}

    for key, value in equation_selection_in.items():
        try:
            num_value = int(value)
            equation_selection[key] = num_value
        except ValueError:
            print(f"Unable to convert '{value}' to an integer for key '{key}'")
    
    del(equation_selection_in)
    Scrf_CP_g = nd.calculate_Scrf_CP_g(animal_input['An_StatePhys'],
                                       animal_input['An_BW'])
    CPGain_FrmGain = nd.calculate_CPGain_FrmGain(animal_input['An_BW'], 
                                                 animal_input['An_BW_mature'])
    NPGain_FrmGain = nd.calculate_NPGain_FrmGain(CPGain_FrmGain,
                                                 nd.coeff_dict)
    Frm_Gain = nd.calculate_Frm_Gain(animal_input['Trg_FrmGain'])
    Rsrv_Gain = nd.calculate_Rsrv_Gain(animal_input['Trg_RsrvGain'])
    Rsrv_Gain_empty = nd.calculate_Rsrv_Gain_empty(Rsrv_Gain)
    NPGain_RsrvGain = nd.calculate_NPGain_RsrvGain(nd.coeff_dict)
    Rsrv_NPgain = nd.calculate_Rsrv_NPgain(NPGain_RsrvGain,
                                           Rsrv_Gain_empty)

    # if animal_input['An_StatePhys'] != 'Lactating Cow':
    #     animal_input['Trg_MilkProd'] = None

    ############################################################################################
    # Start of refactored NASEM 
    ############################################################################################

    ########################################
    # Step 2: DMI Equations
    ########################################
    # pre calculations for DMI:
    ###########################
    # Calculate Target milk net energy
    Trg_NEmilk_Milk = nd.calculate_Trg_NEmilk_Milk(animal_input['Trg_MilkTPp'],
                                                animal_input['Trg_MilkFatp'],
                                                animal_input['Trg_MilkLacp'])
    
    # # Need to precalculate Dt_NDF for DMI predicitons, this will be based on the user entered DMI (animal_input['DMI])
    Dt_NDF = (diet_info_initial['Fd_NDF'] * diet_info_initial['Fd_DMInp']).sum()

    # Predict DMI for heifers
    Kb_LateGest_DMIn = nd.calculate_Kb_LateGest_DMIn(Dt_NDF)
    An_PrePartWklim = nd.calculate_An_PrePartWklim(animal_input['An_PrePartWk'])
    An_PrePartWkDurat = An_PrePartWklim * 2


    if equation_selection['DMIn_eqn'] == 0:
        # print('Using user input DMI')
        pass

    # Predict DMI for lactating cow
    elif equation_selection['DMIn_eqn'] == 8:
        # print("using DMIn_eqn: 8")
        animal_input['DMI'] = nd.calculate_Dt_DMIn_Lact1(
            animal_input['Trg_MilkProd'],
            animal_input['An_BW'],
            animal_input['An_BCS'],
            animal_input['An_LactDay'],
            animal_input['An_Parity_rl'],
            Trg_NEmilk_Milk
            )

    # Individual Heifer DMI Predictions
    elif equation_selection['DMIn_eqn'] in [2, 3, 4, 5, 6, 7]:
        Dt_DMIn_BW_LateGest_i = nd.calculate_Dt_DMIn_BW_LateGest_i(An_PrePartWklim, Kb_LateGest_DMIn, coeff_dict)

        # All the individual DMI predictions require this value
        Dt_DMIn_Heif_LateGestInd = nd.calculate_Dt_DMIn_Heif_LateGestInd(animal_input['An_BW'], Dt_DMIn_BW_LateGest_i)

        if equation_selection['DMIn_eqn'] == 2:
            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_NRCa(animal_input['An_BW'], animal_input['An_BW_mature']), 
                    Dt_DMIn_Heif_LateGestInd
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_NRCa(animal_input['An_BW'], animal_input['An_BW_mature'])

        if equation_selection['DMIn_eqn'] == 3:
            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_NRCad(animal_input['An_BW'], animal_input['An_BW_mature'], Dt_NDF),
                    Dt_DMIn_Heif_LateGestInd
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_NRCad(animal_input['An_BW'], animal_input['An_BW_mature'], Dt_NDF)

        if equation_selection['DMIn_eqn'] == 4:
            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_H1(animal_input['An_BW']),
                    Dt_DMIn_Heif_LateGestInd
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_H1(animal_input['An_BW'])

        if equation_selection['DMIn_eqn'] == 5:
            Dt_NDFdev_DMI = nd.calculate_Dt_NDFdev_DMI(animal_input['An_BW'], Dt_NDF)

            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_H2(animal_input['An_BW'], Dt_NDFdev_DMI),
                    Dt_DMIn_Heif_LateGestInd
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_H2(animal_input['An_BW'],Dt_NDFdev_DMI)

        if equation_selection['DMIn_eqn'] == 6:
            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_HJ1(animal_input['An_BW']),
                    Dt_DMIn_Heif_LateGestInd
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_HJ1(animal_input['An_BW'])

        if equation_selection['DMIn_eqn'] == 7:
            Dt_NDFdev_DMI = nd.calculate_Dt_NDFdev_DMI(animal_input['An_BW'],Dt_NDF)

            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_HJ2(animal_input['An_BW'],Dt_NDFdev_DMI),
                    Dt_DMIn_Heif_LateGestInd
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_HJ2(animal_input['An_BW'], Dt_NDFdev_DMI)

    # Group Heifer DMI Predictions
    elif equation_selection['DMIn_eqn'] in [12, 13, 14, 15, 16, 17]:
        Dt_DMIn_BW_LateGest_p = nd.calculate_Dt_DMIn_BW_LateGest_p(An_PrePartWkDurat, Kb_LateGest_DMIn, coeff_dict)

        # All group DMI predicitons require this value
        Dt_DMIn_Heif_LateGestPen = nd.calculate_Dt_DMIn_Heif_LateGestPen(animal_input['An_BW'], Dt_DMIn_BW_LateGest_p)

        if equation_selection['DMIn_eqn'] == 12:
            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_NRCa(animal_input['An_BW'], animal_input['An_BW_mature']),
                    Dt_DMIn_Heif_LateGestPen
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_NRCa(animal_input['An_BW'], animal_input['An_BW_mature'])

        if equation_selection['DMIn_eqn'] == 13:
            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_NRCad(animal_input['An_BW'], animal_input['An_BW_mature'], Dt_NDF),
                    Dt_DMIn_Heif_LateGestPen
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_NRCad(animal_input['An_BW'], animal_input['An_BW_mature'], Dt_NDF)

        if equation_selection['DMIn_eqn'] == 14:
            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_H1(animal_input['An_BW']), 
                    Dt_DMIn_Heif_LateGestPen
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_H1(animal_input['An_BW'])

        if equation_selection['DMIn_eqn'] == 15:
            Dt_NDFdev_DMI = nd.calculate_Dt_NDFdev_DMI(animal_input['An_BW'],Dt_NDF)

            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_H2(animal_input['An_BW'], Dt_NDFdev_DMI),
                    Dt_DMIn_Heif_LateGestPen
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_H2(animal_input['An_BW'],Dt_NDFdev_DMI)

        if equation_selection['DMIn_eqn'] == 16:
            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_HJ1(animal_input['An_BW']), 
                    Dt_DMIn_Heif_LateGestPen
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_HJ1(animal_input['An_BW'])

        if equation_selection['DMIn_eqn'] == 17:
            Dt_NDFdev_DMI = nd.calculate_Dt_NDFdev_DMI(animal_input['An_BW'],Dt_NDF)

            if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
                animal_input['DMI'] = min(
                    nd.calculate_Dt_DMIn_Heif_HJ2(animal_input['An_BW'],Dt_NDFdev_DMI),
                    Dt_DMIn_Heif_LateGestPen
                    )
            else:
                animal_input['DMI'] = nd.calculate_Dt_DMIn_Heif_HJ2(animal_input['An_BW'], Dt_NDFdev_DMI)

    elif equation_selection['DMIn_eqn'] == 10:
        Dt_DMIn_BW_LateGest_i = nd.calculate_Dt_DMIn_BW_LateGest_i(An_PrePartWklim, Kb_LateGest_DMIn, coeff_dict)

        Dt_DMIn_BW_LateGest_p = nd.calculate_Dt_DMIn_BW_LateGest_p(An_PrePartWkDurat, Kb_LateGest_DMIn, coeff_dict)

        if animal_input['An_PrePartWk'] > An_PrePartWkDurat:
            animal_input['DMI'] = min(
                nd.calculate_Dt_DMIn_DryCow1_FarOff(animal_input['An_BW'], Dt_DMIn_BW_LateGest_i),
                nd.calculate_Dt_DMIn_DryCow1_Close(animal_input['An_BW'], Dt_DMIn_BW_LateGest_p)
                )
        else:
            animal_input['DMI'] = nd.calculate_Dt_DMIn_DryCow1_FarOff(animal_input['An_BW'], Dt_DMIn_BW_LateGest_i)

    elif equation_selection['DMIn_eqn'] == 11:
        animal_input['DMI'] = nd.calculate_Dt_DMIn_DryCow2(animal_input['An_BW'], animal_input['An_GestDay'], animal_input['An_GestLength'])

    else:
        # It needs to catch all possible solutions, otherwise it's possible that it stays unchanged without warning
        print("DMIn_eqn uncaught - DMI not changed. equation_selection[DMIn_eqn]: " + str(equation_selection['DMIn_eqn']))

    # Calculated again as part of diet_data, value may change depending on DMIn_eqn selections
    del (Dt_NDF)

    ########################################
    # Step 3: Feed Based Calculations
    ########################################
    # Calculate An_DMIn_BW with the final DMI value; required for calculating diet_data_initial
    An_DMIn_BW = nd.calculate_An_DMIn_BW(animal_input['An_BW'],
                                         animal_input['DMI'])
    Fe_rOMend = nd.calculate_Fe_rOMend(animal_input['DMI'],
                                       coeff_dict)

    diet_info = nd.calculate_diet_info(animal_input['DMI'],
                                       animal_input['An_StatePhys'],
                                       equation_selection['Use_DNDF_IV'],
                                       diet_info=diet_info_initial,
                                       coeff_dict=coeff_dict)
    # All equations in the f dataframe go into calculate_diet_info()
    # This includes micronutrient calculations which are no longer handled by seperate functions

    diet_data_initial = nd.calculate_diet_data_initial(diet_info,
                                                       animal_input['DMI'],
                                                       animal_input['An_BW'],
                                                       animal_input['An_StatePhys'],
                                                       An_DMIn_BW,
                                                       animal_input['An_AgeDryFdStart'],
                                                       animal_input['Env_TempCurr'],
                                                       equation_selection['DMIn_eqn'],
                                                       Fe_rOMend,
                                                       coeff_dict)
    # diet_data contains everything starting with "Dt_"

    ########################################
    # Step 4: Infusion Calculations
    ########################################

    infusion_data = nd.calculate_infusion_data(infusion_input, animal_input['DMI'], coeff_dict)

########################################
# # Step 4.1: Uterine Bodyweight Calculations
# ########################################
    Uter_Wtpart = nd.calculate_Uter_Wtpart(animal_input['Fet_BWbrth'], coeff_dict)
    Uter_Wt = nd.calculate_Uter_Wt(animal_input['An_Parity_rl'],
                                   animal_input['An_AgeDay'],
                                   animal_input['An_LactDay'],
                                   animal_input['An_GestDay'],
                                   animal_input['An_GestLength'],
                                   Uter_Wtpart,
                                   coeff_dict)
                                   
    GrUter_Wtpart = nd.calculate_GrUter_Wtpart(animal_input['Fet_BWbrth'], coeff_dict)
    GrUter_Wt = nd.calculate_GrUter_Wt(animal_input['An_GestDay'],
                                       animal_input['An_GestLength'],
                                       Uter_Wt,
                                       GrUter_Wtpart,
                                       coeff_dict)

    Uter_BWgain = nd.calculate_Uter_BWgain(animal_input['An_LactDay'],
                                           animal_input['An_GestDay'],
                                           animal_input['An_GestLength'],
                                           Uter_Wt,
                                           coeff_dict)

    GrUter_BWgain = nd.calculate_GrUter_BWgain(animal_input['An_LactDay'],
                                               animal_input['An_GestDay'],
                                               animal_input['An_GestLength'],
                                               GrUter_Wt,
                                               Uter_BWgain,
                                               coeff_dict)

    ########################################
    # Step 5: Animal Level Calculations
    ########################################
    # Combine Diet and Infusion nutrient supplies
    An_data_initial = nd.calculate_An_data_initial(animal_input,
                                                   diet_data_initial,
                                                   infusion_data,
                                                   equation_selection['Monensin_eqn'],
                                                   GrUter_Wt,
                                                   coeff_dict)

    ########################################
    # Step 6: Rumen Digestion Calculations
    ########################################
    # Rumen Digestability Coefficients
    Rum_dcNDF = nd.calculate_Rum_dcNDF(animal_input['DMI'],
                                       diet_data_initial['Dt_NDFIn'],
                                       diet_data_initial['Dt_StIn'],
                                       diet_data_initial['Dt_CPIn'],
                                       diet_data_initial['Dt_ADFIn'],
                                       diet_data_initial['Dt_ForWet'])

    Rum_dcSt = nd.calculate_Rum_dcSt(animal_input['DMI'],
                                     diet_data_initial['Dt_ForNDF'],
                                     diet_data_initial['Dt_StIn'],
                                     diet_data_initial['Dt_ForWet'])

    # Rumen Digestable Intakes
    Rum_DigNDFIn = nd.calculate_Rum_DigNDFIn(Rum_dcNDF,
                                             diet_data_initial['Dt_NDFIn'])

    Rum_DigStIn = nd.calculate_Rum_DigStIn(Rum_dcSt,
                                           diet_data_initial['Dt_StIn'])

    ########################################
    # Step 7: Microbial Protein Calculations
    ########################################
    if equation_selection['MiN_eqn'] == 1:
        RDPIn_MiNmax = nd.calculate_RDPIn_MiNmax(animal_input['DMI'],
                                                 An_data_initial['An_RDP'],
                                                 An_data_initial['An_RDPIn'])
        
        MiN_Vm = nd.calculate_MiN_Vm(RDPIn_MiNmax, coeff_dict)
        
        Du_MiN_g = nd.calculate_Du_MiN_NRC2021_g(MiN_Vm,
                                                 Rum_DigNDFIn,
                                                 Rum_DigStIn,
                                                 An_data_initial['An_RDPIn_g'],
                                                 coeff_dict)
    elif equation_selection['MiN_eqn'] == 2:
        Du_MiN_g = nd.calculate_Du_MiN_VTln_g(diet_data_initial['Dt_rOMIn'],
                                              diet_data_initial['Dt_ForNDFIn'],
                                              An_data_initial['An_RDPIn'],
                                              Rum_DigStIn,
                                              Rum_DigNDFIn,
                                              coeff_dict)
    elif equation_selection['MiN_eqn'] == 3:
        Du_MiN_g = nd.calculate_Du_MiN_VTnln_g(An_data_initial['An_RDPIn'],
                                               Rum_DigNDFIn,
                                               Rum_DigStIn)
    else:
        raise ValueError(
            f"Invalid MiN_eqn: {equation_selection['MiN_eqn']} was entered. Must choose 1, 2 or 3.")

    Du_MiCP_g = nd.calculate_Du_MiCP_g(Du_MiN_g)
    Du_MiTP_g = nd.calculate_Du_MiTP_g(Du_MiCP_g, coeff_dict)
    Du_MiCP = nd.calculate_Du_MiCP(Du_MiCP_g)
    Du_idMiCP_g = nd.calculate_Du_idMiCP_g(Du_MiCP_g,
                                           coeff_dict)
    Du_idMiCP = nd.calculate_Du_idMiCP(Du_idMiCP_g)

    ########################################
    # Step 7.1: Fe_CP Calculation /  Finish Dt_ and An_ calculations
    ########################################
    # Required to finish Dt_ and An_ calculations
    Fe_RUP = nd.calculate_Fe_RUP(An_data_initial['An_RUPIn'],
                                 infusion_data['InfSI_TPIn'],
                                 An_data_initial['An_idRUPIn'])
    Fe_RumMiCP = nd.calculate_Fe_RumMiCP(Du_MiCP,
                                         Du_idMiCP)
    
    Fe_CPend_g = nd.calculate_Fe_CPend_g(animal_input['An_StatePhys'],
                                         An_data_initial['An_DMIn'],
                                         An_data_initial['An_NDF'],
                                         animal_input['DMI'],
                                         diet_data_initial['Dt_DMIn_ClfLiq'],
                                         equation_selection["NonMilkCP_ClfLiq"])
    
    Fe_CPend = nd.calculate_Fe_CPend(Fe_CPend_g)
    Fe_CP = nd.calculate_Fe_CP(animal_input['An_StatePhys'],
                               diet_data_initial['Dt_CPIn_ClfLiq'],
                               diet_data_initial['Dt_dcCP_ClfDry'],
                               An_data_initial['An_CPIn'],
                               Fe_RUP,
                               Fe_RumMiCP,
                               Fe_CPend,
                               infusion_data['InfSI_NPNCPIn'],
                               coeff_dict)

########################################
# Step 7.2: Microbial Amino Acid Calculations
########################################
    AA_list = ['Arg', 'His', 'Ile', 'Leu',
               'Lys', 'Met', 'Phe', 'Thr', 'Trp', 'Val']
    AA_values = pd.DataFrame(index=AA_list)
    # Dataframe for storing all individual amino acid values

    AA_values['Du_AAMic'] = nd.calculate_Du_AAMic(Du_MiTP_g,
                                                  AA_list,
                                                  nd.coeff_dict)
    AA_values['Du_IdAAMic'] = nd.calculate_Du_IdAAMic(AA_values['Du_AAMic'],
                                                      nd.coeff_dict)

    ########################################
    # Step 7.3: Complete diet_data and An_data
    ########################################
    diet_data = nd.calculate_diet_data_complete(diet_data_initial,
                                                animal_input['An_StatePhys'],
                                                Fe_CP,
                                                equation_selection['Monensin_eqn'],
                                                AA_values['Du_IdAAMic'],
                                                coeff_dict)
    An_data = nd.calculate_An_data_complete(An_data_initial,
                                            diet_data,
                                            animal_input['An_StatePhys'],
                                            Fe_CP,
                                            infusion_data,
                                            equation_selection['Monensin_eqn'],
                                            coeff_dict)

########################################
# Step 10: Metabolizable Protein Intake
########################################
    Du_idMiTP_g = nd.calculate_Du_idMiTP_g(Du_idMiCP_g,
                                           nd.coeff_dict)
    Du_idMiTP = nd.calculate_Du_idMiTP(Du_idMiTP_g)
    An_MPIn = nd.calculate_An_MPIn(diet_data['Dt_idRUPIn'],
                                   Du_idMiTP)
    An_MPIn_g = nd.calculate_An_MPIn_g(An_MPIn)

    ########################################
    # Step 8: Amino Acid Calculations
    ########################################

    AA_values['Abs_AA_g'] = nd.calculate_Abs_AA_g(AA_list,
                                                  An_data,
                                                  infusion_data,
                                                  infusion_data['Inf_Art'])

    AA_values['mPrtmx_AA'] = nd.calculate_mPrtmx_AA(AA_list,
                                                    coeff_dict)
    f_mPrt_max = nd.calculate_f_mPrt_max(animal_input['An_305RHA_MlkTP'],
                                         coeff_dict)
    AA_values['mPrtmx_AA2'] = nd.calculate_mPrtmx_AA2(AA_values['mPrtmx_AA'],
                                                      f_mPrt_max)
    AA_values['AA_mPrtmx'] = nd.calculate_AA_mPrtmx(AA_list,
                                                    coeff_dict)
    AA_values['mPrt_AA_01'] = nd.calculate_mPrt_AA_01(AA_values['AA_mPrtmx'],
                                                      AA_list,
                                                      coeff_dict)
    AA_values['mPrt_k_AA'] = nd.calculate_mPrt_k_AA(AA_values['mPrtmx_AA2'],
                                                    AA_values['mPrt_AA_01'],
                                                    AA_values['AA_mPrtmx'])

    Abs_EAA_g = nd.calculate_Abs_EAA_g(AA_values['Abs_AA_g'])
    Abs_neAA_g = nd.calculate_Abs_neAA_g(An_MPIn_g,
                                         Abs_EAA_g)
    Abs_OthAA_g = nd.calculate_Abs_OthAA_g(Abs_neAA_g,
                                           AA_values['Abs_AA_g'])
    Abs_EAA2b_g = nd.calculate_Abs_EAA2b_g(equation_selection['mPrt_eqn'],
                                           AA_values['Abs_AA_g'])
    mPrt_k_EAA2 = nd.calculate_mPrt_k_EAA2(AA_values.loc['Met', 'mPrtmx_AA2'],
                                           AA_values.loc['Met', 'mPrt_AA_01'],
                                           AA_values.loc['Met', 'AA_mPrtmx'])


########################################
# Step 11: Milk Production Prediciton
########################################
    Mlk_NP_g = nd.calculate_Mlk_NP_g(animal_input['An_StatePhys'],
                                     animal_input['An_BW'],
                                     AA_values['Abs_AA_g'],
                                     AA_values['mPrt_k_AA'],
                                     Abs_neAA_g,
                                     Abs_OthAA_g,
                                     Abs_EAA2b_g,
                                     mPrt_k_EAA2,
                                     An_data['An_DigNDF'],
                                     An_data['An_DEInp'],
                                     An_data['An_DEStIn'],
                                     An_data['An_DEFAIn'],
                                     An_data['An_DErOMIn'],
                                     An_data['An_DENDFIn'],
                                     coeff_dict)
                                     
    Mlk_CP_g = nd.calculate_Mlk_CP_g(Mlk_NP_g)
        
    ########################################
    # Step 12: Body Frame and Reserve Gain
    ########################################
    Frm_Gain_empty = nd.calculate_Frm_Gain_empty(Frm_Gain,
                                                diet_data['Dt_DMIn_ClfLiq'],
                                                diet_data['Dt_DMIn_ClfStrt'], 
                                                coeff_dict)

    Body_Gain_empty = nd.calculate_Body_Gain_empty(Frm_Gain_empty, Rsrv_Gain_empty)

    An_REgain_Calf = nd.calculate_An_REgain_Calf(Body_Gain_empty, An_data['An_BW_empty'])

    Frm_NPgain = nd.calculate_Frm_NPgain(animal_input['An_StatePhys'],
                                        NPGain_FrmGain,
                                        Frm_Gain_empty,
                                        Body_Gain_empty,
                                        An_REgain_Calf)

    Body_NPgain = nd.calculate_Body_NPgain(Frm_NPgain, Rsrv_NPgain)
    Body_CPgain = nd.calculate_Body_CPgain(Body_NPgain, coeff_dict)
    Body_CPgain_g = nd.calculate_Body_CPgain_g(Body_CPgain)


########################################
# Step 13: Gestation Protein Use
########################################
    Gest_NCPgain_g = nd.calculate_Gest_NCPgain_g(GrUter_BWgain,coeff_dict)

    Gest_NPgain_g = nd.calculate_Gest_NPgain_g(Gest_NCPgain_g, coeff_dict)

    Gest_NPuse_g = nd.calculate_Gest_NPuse_g(Gest_NPgain_g, coeff_dict)


    Gest_CPuse_g = nd.calculate_Gest_CPuse_g(Gest_NPuse_g, coeff_dict)
    
########################################
# Step 14: Urinary Loss
########################################
    Ur_Nout_g = nd.calculate_Ur_Nout_g(diet_data['Dt_CPIn'],
                                    Fe_CP,
                                    Scrf_CP_g,
                                    Fe_CPend_g,
                                    Mlk_CP_g,
                                    Body_CPgain_g,
                                    Gest_CPuse_g)

    Ur_DEout = nd.calculate_Ur_DEout(Ur_Nout_g)

########################################
# Step 15: Energy Intake
########################################
    An_MEIn = nd.calculate_An_MEIn(animal_input['An_StatePhys'],
                                animal_input['An_BW'],
                                An_data['An_DEIn'],
                                An_data['An_GasEOut'],
                                Ur_DEout,
                                diet_data['Dt_DMIn_ClfLiq'],
                                diet_data['Dt_DEIn_base_ClfLiq'],
                                diet_data['Dt_DEIn_base_ClfDry'],
                                equation_selection['RumDevDisc_Clf'])
    An_NEIn = nd.calculate_An_NEIn(An_MEIn)
    An_NE = nd.calculate_An_NE(An_NEIn,An_data['An_DMIn'])

########################################
# Step 16: Energy Requirement
########################################
    # Maintenance Requirements
    An_NEmUse_NS = nd.calculate_An_NEmUse_NS(animal_input['An_StatePhys'],
                                             animal_input['An_BW'],
                                             An_data['An_BW_empty'],
                                             animal_input['An_Parity_rl'],
                                             diet_data['Dt_DMIn_ClfLiq'])
    
    An_NEm_Act_Graze = nd.calculate_An_NEm_Act_Graze(diet_data['Dt_PastIn'],
                                                     animal_input['DMI'],
                                                     diet_data['Dt_PastSupplIn'],
                                                     An_data['An_MBW'])

    An_NEm_Act_Parlor = nd.calculate_An_NEm_Act_Parlor(animal_input['An_BW'],
                                                       animal_input['Env_DistParlor'],
                                                       animal_input['Env_TripsParlor'])

    An_NEm_Act_Topo = nd.calculate_An_NEm_Act_Topo(animal_input['An_BW'],
                                                   animal_input['Env_Topo'])

    An_NEmUse_Act = nd.calculate_An_NEmUse_Act(An_NEm_Act_Graze,
                                               An_NEm_Act_Parlor,
                                               An_NEm_Act_Topo)

    An_NEmUse = nd.calculate_An_NEmUse(An_NEmUse_NS,
                                       An_NEmUse_Act,
                                       coeff_dict)

    An_MEmUse = nd.calculate_An_MEmUse(An_NEmUse,
                                       coeff_dict)

    # Gain Requirements
    Rsrv_Gain = nd.calculate_Rsrv_Gain(animal_input['Trg_RsrvGain'])

    Rsrv_Gain_empty = nd.calculate_Rsrv_Gain_empty(Rsrv_Gain)

    Rsrv_Fatgain = nd.calculate_Rsrv_Fatgain(Rsrv_Gain_empty,
                                             coeff_dict)

    CPGain_FrmGain = nd.calculate_CPGain_FrmGain(animal_input['An_BW'],
                                                 animal_input['An_BW_mature'])

    Rsrv_CPgain = nd.calculate_Rsrv_CPgain(CPGain_FrmGain,
                                           Rsrv_Gain_empty)

    Rsrv_NEgain = nd.calculate_Rsrv_NEgain(Rsrv_Fatgain,
                                           Rsrv_CPgain)

    Kr_ME_RE = nd.calculate_Kr_ME_RE(animal_input['Trg_MilkProd'],
                                     animal_input['Trg_RsrvGain'])

    Rsrv_MEgain = nd.calculate_Rsrv_MEgain(Rsrv_NEgain,
                                           Kr_ME_RE)

    FatGain_FrmGain = nd.calculate_FatGain_FrmGain(animal_input['An_StatePhys'],
                                                   An_REgain_Calf,
                                                   animal_input['An_BW'],
                                                   animal_input['An_BW_mature'])

    Frm_Gain = nd.calculate_Frm_Gain(animal_input['Trg_FrmGain'])

    Frm_Gain_empty = nd.calculate_Frm_Gain_empty(Frm_Gain,
                                                 diet_data['Dt_DMIn_ClfLiq'],
                                                 diet_data['Dt_DMIn_ClfStrt'],
                                                 coeff_dict)

    Frm_Fatgain = nd.calculate_Frm_Fatgain(FatGain_FrmGain,
                                           Frm_Gain_empty)

    NPGain_FrmGain = nd.calculate_NPGain_FrmGain(CPGain_FrmGain,
                                                 coeff_dict)

    Frm_NPgain = nd.calculate_Frm_NPgain(animal_input['An_StatePhys'],
                                         NPGain_FrmGain,
                                         Frm_Gain_empty,
                                         Body_Gain_empty,
                                         An_REgain_Calf)

    Frm_CPgain = nd.calculate_Frm_CPgain(Frm_NPgain,
                                         coeff_dict)

    Frm_NEgain = nd.calculate_Frm_NEgain(Frm_Fatgain,
                                         Frm_CPgain)

    Frm_MEgain = nd.calculate_Frm_MEgain(Frm_NEgain,
                                         coeff_dict)

    An_MEgain = nd.calculate_An_MEgain(Rsrv_MEgain,
                                       Frm_MEgain)

    # Gestation Requirement
    Gest_REgain = nd.calculate_Gest_REgain(GrUter_BWgain,
                                           coeff_dict)

    Gest_MEuse = nd.calculate_Gest_MEuse(Gest_REgain)

    # Milk Production Requirement
    Trg_NEmilk_Milk = nd.calculate_Trg_NEmilk_Milk(animal_input['Trg_MilkFatp'],
                                                   animal_input['Trg_MilkTPp'],
                                                   animal_input['Trg_MilkLacp'])

    Trg_Mlk_NEout = nd.calculate_Trg_Mlk_NEout(animal_input['Trg_MilkProd'],
                                               Trg_NEmilk_Milk)

    Trg_Mlk_MEout = nd.calculate_Trg_Mlk_MEout(Trg_Mlk_NEout,
                                               coeff_dict)

    # Total Metabolizalbe Energy Requirement
    Trg_MEuse = nd.calculate_Trg_MEuse(An_MEmUse,
                                       An_MEgain,
                                       Gest_MEuse,
                                       Trg_Mlk_MEout)

    ########################################
    # Step 17: Protein Requirement
    ########################################
    # Maintenance Requirement
    Fe_NPend = nd.calculate_Fe_NPend(Fe_CPend)
    
    Fe_NPend_g = nd.calculate_Fe_NPend_g(Fe_NPend)
    
    Fe_MPendUse_g_Trg = nd.calculate_Fe_MPendUse_g_Trg(animal_input['An_StatePhys'],
                                                       Fe_CPend_g,
                                                       Fe_NPend_g,
                                                       coeff_dict)
       
    Scrf_NP_g = nd.calculate_Scrf_NP_g(Scrf_CP_g,
                                       coeff_dict)
        
    Scrf_MPUse_g_Trg = nd.calculate_Scrf_MPUse_g_Trg(animal_input['An_StatePhys'],
                                                     Scrf_CP_g,
                                                     Scrf_NP_g,
                                                     coeff_dict)
        
    Ur_Nend_g = nd.calculate_Ur_Nend_g(animal_input['An_BW'])
    
    Ur_NPend_g = nd.calculate_Ur_NPend_g(Ur_Nend_g)
    
    Ur_MPendUse_g = nd.calculate_Ur_MPendUse_g(Ur_NPend_g)
    
    An_MPm_g_Trg = nd.calculate_An_MPm_g_Trg(Fe_MPendUse_g_Trg,
                                             Scrf_MPUse_g_Trg,
                                             Ur_MPendUse_g)
    
    # Gain Requirement
    Body_NPgain_g = nd.calculate_Body_NPgain_g(Body_NPgain)
    
    Body_MPUse_g_Trg = nd.calculate_Body_MPUse_g_Trg_initial(Body_NPgain_g,
                                                             coeff_dict)

    # Gestation Requirement
    Gest_MPUse_g_Trg = nd.calculate_Gest_MPUse_g_Trg(Gest_NPuse_g,
                                                     coeff_dict)
    
    # Milk Requirement
    Trg_Mlk_NP_g = nd.calculate_Trg_Mlk_NP_g(animal_input['Trg_MilkProd'],
                                             animal_input['Trg_MilkTPp'])
    
    Mlk_MPUse_g_Trg = nd.calculate_Mlk_MPUse_g_Trg(Trg_Mlk_NP_g,
                                                    coeff_dict)

    # Total Protein Requirement
    # NOTE This initial protein requirement is the final value in most cases. There is an
    #      adjustment made when An_StatePhys == "Heifer" and Diff_MPuse_g > 0
    #      Some of the values used to make this adjustment are used elsewhere so it can't just 
    #      be put behind an if statement 
    An_MPuse_g_Trg = nd.calculate_An_MPuse_g_Trg_initial(An_MPm_g_Trg,
                                                         Body_MPUse_g_Trg,
                                                         Gest_MPUse_g_Trg,
                                                         Mlk_MPUse_g_Trg)
    
    An_MEIn_approx = nd.calculate_An_MEIn_approx(An_data['An_DEInp'],
                                                 An_data['An_DENPNCPIn'],
                                                 An_data['An_DigTPaIn'],
                                                 Body_NPgain,
                                                 An_data['An_GasEOut'],
                                                 coeff_dict)
    
    Min_MPuse_g = nd.calculate_Min_MPuse_g(animal_input['An_StatePhys'],
                                           An_MPuse_g_Trg,
                                           animal_input['An_BW'],
                                           animal_input['An_BW_mature'],
                                           An_MEIn_approx)
    
    Diff_MPuse_g = nd.calculate_Diff_MPuse_g(Min_MPuse_g,
                                             An_MPuse_g_Trg)
    
    Frm_NPgain_g = nd.calculate_Frm_NPgain_g(Frm_NPgain)
    
    An_BWmature_empty = nd.calculate_An_BWmature_empty(animal_input['An_BW_mature'],
                                                       coeff_dict)
    

    Kg_MP_NP_Trg = nd.calculate_Kg_MP_NP_Trg(animal_input['An_StatePhys'],
                                             animal_input['An_Parity_rl'],
                                             animal_input['An_BW'],
                                             An_data['An_BW_empty'],
                                             animal_input['An_BW_mature'],
                                             An_BWmature_empty,
                                             MP_NP_efficiency_input,
                                             coeff_dict)

    Frm_MPUse_g_Trg = nd.calculate_Frm_MPUse_g_Trg(animal_input['An_StatePhys'],
                                                   Frm_NPgain_g,
                                                   Kg_MP_NP_Trg,
                                                   Diff_MPuse_g)
            
    Rsrv_NPgain_g = nd.calculate_Rsrv_NPgain_g(Rsrv_NPgain)
    
    Rsrv_MPUse_g_Trg = nd.calculate_Rsrv_MPUse_g_Trg(animal_input['An_StatePhys'],
                                                     Diff_MPuse_g,
                                                     Rsrv_NPgain_g,
                                                     Kg_MP_NP_Trg)
    
    # Recalculate 
    Body_MPUse_g_Trg = nd.calculate_Body_MPUse_g_Trg(animal_input['An_StatePhys'],
                                                     Diff_MPuse_g,
                                                     Body_NPgain_g,
                                                     Body_MPUse_g_Trg,
                                                     Kg_MP_NP_Trg)
    # Recalculate
    An_MPuse_g_Trg = nd.calculate_An_MPuse_g_Trg(An_MPm_g_Trg,
                                                 Frm_MPUse_g_Trg,
                                                 Rsrv_MPUse_g_Trg,
                                                 Gest_MPUse_g_Trg,
                                                 Mlk_MPUse_g_Trg)
    
    ########################################
    # Temporary results export
    ########################################
    # print(Trg_MEuse)

    model_out_dict = {
        'infusion_data' : infusion_data,
        'Du_MiN_g' : Du_MiN_g,
        # testing Monensin:
        'Dt_DEIn': diet_data['Dt_DEIn'],
        'An_DEIn': An_data['An_DEIn']
    }

    return animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict

# 
# ### RUN MODEL ###
animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = dev_run_NASEM_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)


# Run model - different options

In [ ]:
pd.set_option('display.max_columns', 100)

In [5]:
animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = dev_run_NASEM_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)

# diet_info
An_data

60.27268034635927


{'An_RDPIn': 3.5560577798966095,
 'An_RDP': 14.50209118672407,
 'An_RDPIn_g': 3556.0577798966096,
 'An_NDFIn': 8.101661552362442,
 'An_NDF': 33.039686604797694,
 'An_DigNDFIn': 3.776616721309864,
 'An_DENDFIn': 15.86179022950143,
 'An_DigStIn': 4.433372410458015,
 'An_DEStIn': 18.753165296237402,
 'An_DigrOMaIn': 2.9794319462397736,
 'An_DErOMIn': 11.917727784959094,
 'An_idRUPIn': 1.2122681602504046,
 'An_RUPIn': 1.610506642682202,
 'An_DMIn': 24.521,
 'An_CPIn': 5.166564422578811,
 'An_DigNDF': 15.401560789975385,
 'An_GEIn': 105.43479040464584,
 'An_GasEOut': 7.023146715857372,
 'An_GutFill_BW': 0.18,
 'An_BWnp': 624.4413282080761,
 'An_GutFill_Wt': 112.39943907745369,
 'An_BW_empty': 512.3955609225462,
 'An_DigCPaIn': 3.966372959464227,
 'An_DECPIn': 22.410007220972883,
 'An_DENPNCPIn': 0.0,
 'An_DETPIn': 22.410007220972883,
 'An_DigFAIn': 0.42091626282339095,
 'An_DEFAIn': 3.956612870539875,
 'An_DEIn': array(72.8993034),
 'An_DEInp': 50.489296181237805,
 'An_IdArgIn': 137.4578200

## Toggle infusions
Notes: found that infusions are being imported within the function, need to move all file imports out of the run model function.
There is also double up of the logic for use_infusions, so modifying that to check which one to use once. 
It could actually be avoided by just providing a default infusion dictionary if one is not given to function directly.
These changes make the 'use_infusions' flag obsolete

In [5]:
# Test DMI equations
equation_selection_in['use_infusions'] = 0 # FALSE

animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = dev_run_NASEM_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)

# diet_info
# infusion_data


In [6]:
# equation_selection_in['use_infusions'] = 1

animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = dev_run_NASEM_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict,
    infusion_input= infusion_custom)

# diet_info
model_out_dict

{'infusion_data': {'Inf_Acet_g': 3000.0,
  'Inf_ADF_g': 4000.0,
  'Inf_Arg_g': 5000.0,
  'Inf_Ash_g': 6000.0,
  'Inf_Butr_g': 7000.0,
  'Inf_CP_g': 8000.0,
  'Inf_CPARum_CP': 9000.0,
  'Inf_CPBRum_CP': 10000.0,
  'Inf_CPCRum_CP': 11000.0,
  'Inf_dcFA': 12000.0,
  'Inf_dcRUP': 13000.0,
  'Inf_DM_g': 14000.0,
  'Inf_EE_g': 15000.0,
  'Inf_FA_g': 16000.0,
  'Inf_Glc_g': 17000.0,
  'Inf_His_g': 18000.0,
  'Inf_Ile_g': 19000.0,
  'Inf_KdCPB': 20000.0,
  'Inf_Leu_g': 21000.0,
  'Inf_Lys_g': 22000.0,
  'Inf_Met_g': 23000.0,
  'Inf_NDF_g': 24000.0,
  'Inf_NPNCP_g': 25000.0,
  'Inf_Phe_g': 26000.0,
  'Inf_Prop_g': 27000.0,
  'Inf_St_g': 28000.0,
  'Inf_Thr_g': 29000.0,
  'Inf_Trp_g': 30000.0,
  'Inf_ttdcSt': 31000.0,
  'Inf_Val_g': 32000.0,
  'Inf_VFA_g': 33000.0,
  'Inf_Location': nan,
  'Inf_DMIn': 14.0,
  'Inf_StIn': 28.0,
  'Inf_NDFIn': 24.0,
  'Inf_ADFIn': 4.0,
  'Inf_GlcIn': 17.0,
  'Inf_CPIn': 8.0,
  'Inf_NPNCPIn': 25.0,
  'Inf_FAIn': 16.0,
  'Inf_AshIn': 6.0,
  'Inf_VFAIn': 33.0,
  'Inf

## Test DMI equation selections

In [7]:
# Equation = 0 should match user input:
equation_selection_in['DMIn_eqn'] = 0
animal_input_in['DMI'] = 24.521 # same as .csv file

animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = dev_run_NASEM_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)

animal_input['DMI']

24.521

In [8]:
# Equation = 8 should calculate calculate_Dt_DMIn_Lact1()
equation_selection_in['DMIn_eqn'] = 8
animal_input_in['DMI'] = 24.521 # same as .csv file

animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = dev_run_NASEM_model(
    user_diet = user_diet_in, 
    animal_input = animal_input_in, 
    equation_selection = equation_selection_in, 
    feed_library_df = feed_library_in, 
    coeff_dict = nd.coeff_dict)

animal_input['DMI']

21.687577670699824

In [9]:
# make function to test all values
def iterate_test_DMI_eqns(DMIn_eqn):
    equation_selection_in_test = equation_selection_in.copy()
    equation_selection_in_test['DMIn_eqn'] = DMIn_eqn

    animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = dev_run_NASEM_model(
        user_diet = user_diet_in, 
        animal_input = animal_input_in, 
        equation_selection = equation_selection_in_test, 
        feed_library_df = feed_library_in, 
        coeff_dict = nd.coeff_dict)

    return (animal_input['DMI'])

[iterate_test_DMI_eqns(DMIn_eqn_value) for DMIn_eqn_value in range(0, 16)]


DMIn_eqn uncaught - DMI not changed. equation_selection[DMIn_eqn]: 1
DMIn_eqn uncaught - DMI not changed. equation_selection[DMIn_eqn]: 9


[24.521,
 24.521,
 11.504513756424398,
 16.842879099441184,
 11.474631902511609,
 12.805095198359778,
 10.866105707801568,
 12.251816154227694,
 21.687577670699824,
 24.521,
 12.323872879290622,
 12.364693050000001,
 11.504513756424398,
 16.842879099441184,
 11.474631902511609,
 12.805095198359778]

In [ ]:
## Still need to test this condition: 
#if animal_input['An_PrePartWk'] > An_PrePartWkDurat:

print(animal_input['An_PrePartWk'])

# Calculate additional physiology values
An_PrePartDay = animal_input_in['An_GestDay'] - animal_input_in['An_GestLength']
An_PrePartDay / 7
# animal_input_in['An_PrePartWk']

# Test equation_selection['MiN_eqn']

In [10]:
# equation_selection['MiN_eqn']


# make function to test all values
def iterate_test_Du_MiN_g_eqns(MiN_eqn):
    print(MiN_eqn)
    equation_selection_in_test = equation_selection_in.copy()
    equation_selection_in_test['MiN_eqn'] = MiN_eqn

    animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = dev_run_NASEM_model(
        user_diet = user_diet_in, 
        animal_input = animal_input_in, 
        equation_selection = equation_selection_in_test, 
        feed_library_df = feed_library_in, 
        coeff_dict = nd.coeff_dict)

    return (model_out_dict['Du_MiN_g'])

[iterate_test_Du_MiN_g_eqns(DMIn_eqn_value) for DMIn_eqn_value in range(1, 4)]

1
2
3


[299.31825131136395, 122.74827303321794, 276.5276811382165]

In [11]:
equation_selection_in

{'Use_DNDF_IV': 0.0,
 'DMIn_eqn': 8,
 'mProd_eqn': 0.0,
 'MiN_eqn': 1.0,
 'use_infusions': 0,
 'NonMilkCP_ClfLiq': 0.0,
 'Monensin_eqn': 0.0,
 'mPrt_eqn': 0.0}

# test Monensin eqn

In [12]:
equation_selection_in


{'Use_DNDF_IV': 0.0,
 'DMIn_eqn': 8,
 'mProd_eqn': 0.0,
 'MiN_eqn': 1.0,
 'use_infusions': 0,
 'NonMilkCP_ClfLiq': 0.0,
 'Monensin_eqn': 0.0,
 'mPrt_eqn': 0.0}

In [13]:
# make function to test all values
def iterate_test_Mon_eqns(Monensin_eqn):
    print(Monensin_eqn)
    equation_selection_in_test = equation_selection_in.copy()
    equation_selection_in_test['Monensin_eqn'] = Monensin_eqn

    animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data, An_data, model_out_dict = dev_run_NASEM_model(
        user_diet = user_diet_in, 
        animal_input = animal_input_in, 
        equation_selection = equation_selection_in_test, 
        feed_library_df = feed_library_in, 
        coeff_dict = nd.coeff_dict)

    return (model_out_dict['Dt_DEIn'], model_out_dict['An_DEIn'])

[iterate_test_Mon_eqns(Monensin_eqn) for Monensin_eqn in range(0, 2)]

0
1


[(array(69.04770702), array(69.04770702)),
 (array(70.42866116), array(70.42866116))]